# Built-in Modules

- [Item 42: Define Function Decorators with *functools.wraps*](http://localhost:8888/notebooks/Built-in-Modules.ipynb#Item-42:-Define-Function-Decorators-with-functools.wraps)
- [Item 43: Consider *contextlib* and *with* Statements for Resuable try/finally Behavior](#Item-43:-Consider-contextlib-and-with-Statements-for-Resuable-try/finally-Behavior)
- [Item 44: Make *pickle* Reliable with *copyreg*](#Item-44:-Make-pickle-Reliable-with-copyreg)
- [Item 45: Use datetime Instead of time for Local Clocks](#Item-45:-Use-datetime-Instead-of-time-for-Local-Clocks)
- [Item 46: Use Built-in Algorithms and Data Structures](#Item-46:-Use-Built-in-Algorithms-and-Data-Structures)
- [Item 47: Use decimal When Precision Is Paramount](#Item-47:-Use-decimal-When-Precision-Is-Paramount)

## Item 42: Define Function Decorators with *functools.wraps*

In [ ]:
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r' %
              (func.__name__, args, kwargs, result))
        return result
    return wrapper

@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

fibonacci(3)

print(fibonacci)
help(fibonacci)

from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r' %
              (func.__name__, args, kwargs, result))
        return result
    return wrapper

@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

# fibonacci = trace(fibonacci)

print(fibonacci)
help(fibonacci)

### Things to Remember

- Decorators are Python syntax for allowing one function to modify another function at runtime.
- Using decorators can cause strange behaviors in tools that do introspection, such as debuggers.
- Use the **wraps** decorator from the **functools** built-in module when you define your own decorators to avoid any issues.

## Item 43: Consider *contextlib* and *with* Statements for Resuable *try/finally* Behavior

In [ ]:
from threading import Lock

lock = Lock()
with lock:
    print('Lock is held')
    
# equivalent
lock.acquire()
try:
    print('Lock is held')
finally:
    lock.release()

The **with** statement version of this is better because it eliminates the need to write the repetitive code of the **try/finally** construction.

In [ ]:
import logging

def my_function():
    logging.debug('Some debug data')
    logging.error('Error log here')
    logging.debug('More debug data')
    
my_function()

from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield
    finally:
        logger.setLevel(old_level)
        
"""
The yield expression is the point at which 
the with block's contents will execute.
Any exceptions that happen in the with block
with be re-raised by the yield expression for
you to catch in the helper function
"""

with debug_logging(logging.DEBUG):
    logging.info('Inside')
    my_function()
logging.warning('After')
my_function()

In [ ]:
from contextlib import contextmanager

@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)
        
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug('This is my message')
    logging.debug('This will not print')

logger = logging.getLogger('my-log')
logger.debug('Debug will not print')
logger.error('Error will print')

### Things to Remember

- The **with** statement allows you to reuse logic from **try/finally** blocks and reduce visual noise.
- The **contextlib** built-in module provides a **contextmanager** decorator that makes it easy to use your own functions in **with** statements.
- The value yielded by context managers is supplied to the as part of the **with** statement. It's useful for letting your code directly access the cause of the special context.

## Item 44: Make *pickle* Reliable with *copyreg*

The pickle built-in module can serialize Python objects into a stream of bytes and deserialize bytes back into objects.

The **pickle** module's serialization format is unsafe by design. In contrast, the **json** module is safe by design.

In [ ]:
class GameState(object):
    def __init__(self, level=0, lives=4, points=0):
        self.level = level
        self.lives = lives
        self.points = points
        
def unpickle_game_state(kwargs):
    version = kwargs.pop('version', 1)
    if version == 1:
        kwargs.pop('lives')
    return GameState(**kwargs)

def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    kwargs['version'] = 2  # Versioning Classes
    return unpickle_game_state, (kwargs,)

import copyreg, pickle

copyreg.pickle(GameState, pickle_game_state)

state = GameState()
state.points += 1000
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

### Things to Remember

- The *pickle* built-in module is only useful for serializing and deserializing objects between trusted programs.
- The *pickle* module may break down when used for more than trivial use cases.
- Use the *copyreg* built-in module with *pickle* to add missing attribute values, allow versioning of classes, and provide stable import path.

## Item 45: Use *datetime* Instead of *time* for Local Clocks

Coordinated Universal Time (UTC) is the standard, time-zone-independent representation of time. UTC works great for computers that represent time as seconds since the UNIX epoch. But UTC isn't ideal for humans. Humans reference time relative to where they're currently located.

Python provides two ways of accomplishing time zone conversions. The old way, using the *time* built-in module, is disastrously error prone. The new way, using the *datetime* built-in module, works great with some help from the community-buit package named *pytz*.

## Item 46: Use Built-in Algorithms and Data Structures

### Double-ended Queue

In [ ]:
from collections import deque

fifo = deque()
fifo.append(1)      # Producer
x = fifo.popleft()  # Consumer

*deque* provides constant time operations for inserting or removing items from its beginning or end.

For the *list* built-in type, inserting or removing items from the end of a list in constant time, while inserting or removing items from the head of a list takes linear time.

### Ordered Dictionary

Standard dictionaries are unordered. That means a *dict* with the same keys and values can result in different orders of iteration.

The *OrderedDict* class from the *collections* module is a special type of dictionary that keeps track of the order in which its keys were inserted.

In [ ]:
from collections import OrderedDict

a = OrderedDict()
a['bar'] = 1
a['foo'] = 2
b = OrderedDict()
b['foo'] = 'one'
b['bar'] = 'two'

for value1, value2 in zip(a.values(), b.values()):
    print(value1, value2)

### Default Dictionary

One probem with dictionaries is that you can't assume any keys are already present. That makes it clumsy to do simple things like increment a counter stored in a dictionary.

In [ ]:
stats = {}
key = 'my_counter'
if key not in stats:
    stats[key] = 0
stats[key] += 1

print(stats)

The *defaultdict* class from collections module simplifies this by automatically storing a default value when a key doesn't exist. All you have to do is provide a function that will return the default value each time a key is missing.

In [ ]:
from collections import defaultdict

stats = defaultdict(int)
stats['my_counter'] += 1

print(stats)

### Heap Queue

In [ ]:
from heapq import heappush, heappop, nsmallest

a = []
heappush(a, 3)
heappush(a, 7)
heappush(a, 5)

print(nsmallest(2, a))

print(heappop(a), heappop(a), heappop(a))

Each of these *heapq* operations takes logarithmic time in proportion to the length of the list. Doing the same work with a standard Python list would scale linearly.

### Bisection

In [ ]:
from bisect import bisect_left
from timeit import timeit

x = list(range(10**6))
i = x.index(991234)  # linear
i = bisect_left(x, 991234)  # logarithmic

### Iterator Tools

The *itertools* function fall into three main categories:

- Linking iterators together
    - *chain*: Combines multiple iterators into a single sequential iterator.
    - *cycle*: Repeats an iterator's items forever.
    - *tee*: Splits a single iterator into multiple parallel iterators.
    - *zip_longest*: A variant of the *zip* built-in function that works well with iterators of different lengths.
- Filtering items from an iterator
    - *islice*: Slices an iterator by numerical indexes without copying.
    - *takewhile*: Returns items from an iterator while a predicate function return *True*.
    - *dropwhile*: Return items from an iterator once the predicate function return *False* for the first time.
    - *filterfalse*: Return all items from an iterator where a predicate function return *False*. The opposite of the *filter* built-in function.
- Combinations of items from iterators
    - *product*: Returns the Cartesian product of items from an iterator, which is a nice alternative to deeply nested list comprehensions.
    - *permutations*: Returns ordered permutations of length N with items from an iterator.
    - *combination*: Return the unordered combinations of length N with unrepeated items from an iterator.

### Things to Remember

- Use Python-s built-in modules for algorithms and data structures.
- Don't reimplement this functionality yourself. It's hard to get right.

## Item 47: Use *decimal* When Precision Is Paramount

In [ ]:
from decimal import Decimal, ROUND_UP

rate = Decimal('0.005')
seconds = Decimal('5')
cost = rate * seconds / Decimal('60')
print(cost)
rounded = cost.quantize(Decimal('0.01'), rounding=ROUND_UP)
print(rounded)

### Things to Remember

- Python has built-in types and classes in modules that can represent practically every type of numerical value.
- The *Decimal* class is ideal for situations that require high precision and exact rounding behavior, such as computations of monetary values.